<a href="https://colab.research.google.com/github/HermitCrab07/Impact_of_HbA1c_on_Readmission_Rates/blob/main/_1_Creation_of_DiabetesDataset_for_Causal_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



---


I. Description of the Dataset and creation of new variables -

The instances represent hospitalized patient
records diagnosed with diabetes. The dataset represents ten years (1999-2008) of clinical care at 130 US hospitals and integrated delivery networks. It includes over 50 features representing patient and hospital outcomes.

Information was extracted from the database for encounters that satisfied the following criteria.
(1)	It is an inpatient encounter (a hospital admission).
(2)	It is a diabetic encounter, that is, one during which any kind of diabetes was entered into the system as a diagnosis.
(3)	The length of stay was at least 1 day and at most 14 days.
(4)	Laboratory tests were performed during the encounter.
(5)	Medications were administered during the
encounter.

The data contains attributes as patient number, race, gender, age, admission type, time in hospital, medical specialty of admitting physician, number of lab tests performed, HbA1c test result, diagnosis, number of medications, and emergency visits in the year before the hospitalization, etc.

Next Steps - we will define 3 new variables and then save the dataset for use in subsequent analysis:

*   Prepare the dataset for analysis
*   Basic exploratory data analysis
*  Create Insured or Not Insured Group
*   Create Treatment/Control based on if a patient received HbA1c measurement or not
*   Create Readmission to include both Early (<30 days) and Late (>30 days) readmission



---



In [1]:
# Beata Strack, Jonathan P. DeShazo, Chris Gennings, Juan L. Olmo, Sebastian Ventura, Krzysztof J. Cios, and John N. Clore
# Impact of HbA1c Measurement on Hospital Readmission Rates: Analysis of 70,000 Clinical Database Patient Records
# BioMed Research International, vol. 2014, Article ID 781670, 11 pages, 2014.
# Dataset was downloaded from UCI Machine Learning Repository.

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

file_path = '/content/diabetic_data.csv'

# Read the CSV file into a pandas DataFrame
df_full = pd.read_csv(file_path)

# Print the first 5 rows of the DataFrame
# print(df_full.head())

In [3]:
# Patients have more than one encounter - some have as many as 40 encounters
frequency = df_full['patient_nbr'].value_counts()
print(frequency)

patient_nbr
88785891     40
43140906     28
88227540     23
1660293      23
23199021     23
             ..
174477542     1
38726739      1
77391171      1
89869032      1
63555939      1
Name: count, Length: 71518, dtype: int64


In [4]:
# Original dataset is approximately 101k records. On retaining only the first encounter_id for each patient_nbr, we reduce dataset to approx. 71k records.
df_first_encounter = df_full.sort_values(['patient_nbr', 'encounter_id']).groupby('patient_nbr').first().reset_index()
df_first_encounter.describe().style.format("{:.2f}")

,patient_nbr,encounter_id,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses
count,71518.00,71518.00,71518.00,71518.00,71518.00,71518.00,71518.00,71518.00,71518.00,71518.00,71518.00,71518.00,71518.00
mean,55051603.80,156815588.30,2.10,3.59,5.66,4.29,43.08,1.43,15.71,0.28,0.10,0.18,7.25
std,39475727.88,100376881.97,1.51,5.27,4.16,2.95,19.95,1.76,8.31,1.07,0.51,0.60,1.99
min,135.00,12522.00,1.00,1.00,1.00,1.00,1.00,0.00,1.00,0.00,0.00,0.00,1.00
25%,23375767.50,81458068.50,1.00,1.00,1.00,2.00,31.00,0.00,10.00,0.00,0.00,0.00,6.00
50%,48166798.50,144057399.00,1.00,1.00,7.00,3.00,44.00,1.00,14.00,0.00,0.00,0.00,8.00
75%,87522216.75,215551324.50,3.00,3.00,7.00,6.00,57.00,2.00,20.00,0.00,0.00,0.00,9.00
max,189502619.00,443867222.00,8.00,28.00,25.00,14.00,132.00,6.00,81.00,42.00,42.00,12.00,16.00


In [5]:
# What is the percentage missing? Calculate the percentage of missing values for each column
percent_missing = df_first_encounter.isnull().sum() * 100 / len(df_first_encounter)
print(percent_missing)

patient_nbr                  0.000000
encounter_id                 0.000000
race                         0.000000
gender                       0.000000
age                          0.000000
weight                       0.000000
admission_type_id            0.000000
discharge_disposition_id     0.000000
admission_source_id          0.000000
time_in_hospital             0.000000
payer_code                   0.000000
medical_specialty            0.000000
num_lab_procedures           0.000000
num_procedures               0.000000
num_medications              0.000000
number_outpatient            0.000000
number_emergency             0.000000
number_inpatient             0.000000
diag_1                       0.000000
diag_2                       0.000000
diag_3                       0.000000
number_diagnoses             0.000000
max_glu_serum               94.949523
A1Cresult                   78.483738
metformin                    0.000000
repaglinide                  0.000000
nateglinide 

In [30]:
# Reduce the dataset to a smaller set of features - only the key ones I will use for the analysis
columns_to_keep = ['encounter_id', 'patient_nbr', 'race', 'weight', 'age', 'gender',
                  'admission_type_id',
                  'discharge_disposition_id',
                  'admission_source_id',
                  'time_in_hospital',
                  'payer_code',
                  'medical_specialty',
                  'num_lab_procedures',
                  'num_procedures',
                  'num_medications',
                  'number_outpatient',
                  'number_emergency',
                  'number_inpatient',
                  'diag_1',
                  'diag_2',
                  'diag_3',
                  'number_diagnoses',
                  'max_glu_serum',
                  'A1Cresult',
                  'metformin',
                  'insulin',
                  'change',
                  'diabetesMed', 'readmitted']

In [31]:
# Subset based on the above list to retain a smaller set of features
df = df_first_encounter[columns_to_keep]
df.describe()

,encounter_id,patient_nbr,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses
count,7.151800e+04,7.151800e+04,71518.000000,71518.000000,71518.000000,71518.00000,71518.000000,71518.000000,71518.000000,71518.000000,71518.000000,71518.000000,71518.000000
mean,1.568156e+08,5.505160e+07,2.100254,3.593235,5.657457,4.28913,43.075478,1.430577,15.705025,0.280069,0.103540,0.177829,7.245700
std,1.003769e+08,3.947573e+07,1.508301,5.269771,4.162580,2.94921,19.952338,1.759864,8.311163,1.068957,0.509187,0.603790,1.994674
min,1.252200e+04,1.350000e+02,1.000000,1.000000,1.000000,1.00000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000
25%,8.145807e+07,2.337577e+07,1.000000,1.000000,1.000000,2.00000,31.000000,0.000000,10.000000,0.000000,0.000000,0.000000,6.000000
50%,1.440574e+08,4.816680e+07,1.000000,1.000000,7.000000,3.00000,44.000000,1.000000,14.000000,0.000000,0.000000,0.000000,8.000000
75%,2.155513e+08,8.752222e+07,3.000000,3.000000,7.000000,6.00000,57.000000,2.000000,20.000000,0.000000,0.000000,0.000000,9.000000
max,4.438672e+08,1.895026e+08,8.000000,28.000000,25.000000,14.00000,132.000000,6.000000,81.000000,42.000000,42.000000,12.000000,16.000000


In [8]:
# Subset_df has only one observation per patient
frequency = df['patient_nbr'].value_counts()
print(frequency)

patient_nbr
189502619    1
135          1
378          1
729          1
774          1
            ..
6723         1
6696         1
6309         1
6228         1
6174         1
Name: count, Length: 71518, dtype: int64


In [9]:
# Define Treatment and Control - will use A1Cresult - and will assess how many have this measurement first.
# Then will divide the data into 2 groups - one that made the measurement and the other that didn't.

print("Unique values in 'A1Cresult':", df['A1Cresult'].unique())

Unique values in 'A1Cresult': [None '>7' '>8' 'Norm']


In [32]:
df['A1C'] = np.where(df['A1Cresult'].isna(), 'Missing', df['A1Cresult'])
print("Unique values in 'A1C':", df['A1C'].unique())
df['A1C'].isna().sum()

Unique values in 'A1C': ['Missing' '>7' '>8' 'Norm']


<ipython-input-32-03f6db31cd72>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['A1C'] = np.where(df['A1Cresult'].isna(), 'Missing', df['A1Cresult'])


np.int64(0)

In [33]:
# Classify into Treatment and Control - then a crosstab to make sure everything is correct
def assign_group(row):
    if row['A1C'] in ('>7', '>8', 'Norm'):
        return 'Treatment'
    elif row['A1C'] in ('Missing'):
        return 'Control'
    else:
        return 'Control'

df['Group'] = df.apply(assign_group, axis=1)

print("Unique values in 'Group':", df['Group'].unique())

# Create the cross-tabulation
cross_tab = pd.crosstab(df['Group'], df['A1C'])

# Print the crosstab
print(cross_tab)

Unique values in 'Group': ['Control' 'Treatment']
A1C          >7    >8  Missing  Norm
Group                               
Control       0     0    56130     0
Treatment  3484  7304        0  4600


<ipython-input-33-46a07b4fdcc3>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Group'] = df.apply(assign_group, axis=1)


In [34]:
# Significant assumption - if there is no payer_code or it is a question mark - assume there is no insurance.

df['Treatment'] = df['A1C'].apply(lambda x: 0 if x == 'Missing' else 1)
df['Insured'] = df['payer_code'].apply(lambda x:0 if x=='?' else 1)


<ipython-input-34-aad2052001a6>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Treatment'] = df['A1C'].apply(lambda x: 0 if x == 'Missing' else 1)
<ipython-input-34-aad2052001a6>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Insured'] = df['payer_code'].apply(lambda x:0 if x=='?' else 1)


In [35]:
df['Readmission'] = df['readmitted'].map({'NO': 0, '<30': 1, '>30': 1})
df['Readmission'].value_counts()

<ipython-input-35-b57337ffc204>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Readmission'] = df['readmitted'].map({'NO': 0, '<30': 1, '>30': 1})


,count
Readmission,
0,42985
1,28533




---



---



In [36]:
df.to_csv('/content/drive/My Drive/Analysis_Diabetes_Dataset_CausalInference/reanalysis_diabetes_dataset.csv', index=False)

In [37]:
df = pd.read_csv('/content/drive/My Drive/Analysis_Diabetes_Dataset_CausalInference/reanalysis_diabetes_dataset.csv')
df.describe()

,encounter_id,patient_nbr,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,Treatment,Insured,Readmission
count,7.151800e+04,7.151800e+04,71518.000000,71518.000000,71518.000000,71518.00000,71518.000000,71518.000000,71518.000000,71518.000000,71518.000000,71518.000000,71518.000000,71518.000000,71518.000000,71518.000000
mean,1.568156e+08,5.505160e+07,2.100254,3.593235,5.657457,4.28913,43.075478,1.430577,15.705025,0.280069,0.103540,0.177829,7.245700,0.215163,0.565941,0.398962
std,1.003769e+08,3.947573e+07,1.508301,5.269771,4.162580,2.94921,19.952338,1.759864,8.311163,1.068957,0.509187,0.603790,1.994674,0.410938,0.495636,0.489688
min,1.252200e+04,1.350000e+02,1.000000,1.000000,1.000000,1.00000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
25%,8.145807e+07,2.337577e+07,1.000000,1.000000,1.000000,2.00000,31.000000,0.000000,10.000000,0.000000,0.000000,0.000000,6.000000,0.000000,0.000000,0.000000
50%,1.440574e+08,4.816680e+07,1.000000,1.000000,7.000000,3.00000,44.000000,1.000000,14.000000,0.000000,0.000000,0.000000,8.000000,0.000000,1.000000,0.000000
75%,2.155513e+08,8.752222e+07,3.000000,3.000000,7.000000,6.00000,57.000000,2.000000,20.000000,0.000000,0.000000,0.000000,9.000000,0.000000,1.000000,1.000000
max,4.438672e+08,1.895026e+08,8.000000,28.000000,25.000000,14.00000,132.000000,6.000000,81.000000,42.000000,42.000000,12.000000,16.000000,1.000000,1.000000,1.000000
